In [ ]:
import pyomo.environ as pyo

def model():
    m = pyo.ConcreteModel()
    m.x1 = pyo.Var(within=pyo.NonNegativeReals)
    m.x2 = pyo.Var(within=pyo.NonNegativeReals)
    m.cost = pyo.Objective(expr=10*m.x1 + 7*m.x2, sense = pyo.maximize)
    m.c1 = pyo.Constraint(expr=m.x1 + 2 * m.x2 <= 10)
    m.c2 = pyo.Constraint(expr=3 * m.x1 + 4*m.x2 <= 40)
    m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    m.rc = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    return m

Example PL:

$$
\max z = 10 x_1 + 7 x_2 \\
s.t. \\
x_1 + 2 x_2 \le 10 \\
3 x_1 + 4 x_2 \le 40 \\
$$

Let's consider the canonical representation of the problem:
$$
\max z = c^T x \\
s.t. Ax = b
$$

We have: A = \[B, N\], x = (xB xN), c = (cB cN)

$$
\max z = c_B x_B + c_N x_N \\
s.t. B x_B + N x_N = b
$$

We have: $x_B = B^{-1}b - B^{-1} N x_N$

$$
z = c_B x_B + c_N x_N \\
z = c_B (B^{-1}b - B^{-1} N x_N) + c_N x_N \\
z = \underbrace{c_B B^{-1}b}_{\hat{z},\ current\ solution} + \underbrace{(c_N - c_B B^{-1} N)}_{gain,\ reduced\ cost} x_N \\
z = c_B B^{-1}b + (c_N - \pi N) x_N
$$

We have: $\pi = c_B B^{-1}$, the dual variable / shadow price.

In [ ]:
m = model()
solver = pyo.SolverFactory("cbc")
solver.solve(m)
objective = pyo.value(m.cost)

A **shadow price** value is associated with each constraint of the model.
It is the instantaneous change in the objective value of the optimal 
solution obtained by changing the right hand side constraint by one unit.

In [ ]:
shadow_price_c1 = m.dual[m.c1]

A **reduced cost** value is associated with each variable of the model.
It is the amount by which an objective function parameter would have
to improve before it would be possible for a corresponding variable
to assume a positive value in the optimal solution.

x2 is outside the basis, its value is 0. Its reduce cost is negative.

Non-basis variables with reduce cost equal to 0 are alternative optimal solutions.

In [ ]:
assert pyo.value(m.x2) == 0
reduce_cost_x2 = m.rc[m.x2]
assert reduce_cost_x2 <= 0

x1 is in the basis so its value is different from 0 and its reduce cost equals 0

In [ ]:
assert pyo.value(m.x1) != 0
assert m.rc[m.x1] == 0

Slack variable: unused capacity. The constraint c1 is saturated (slack variable associated to c1 = 0) but not c2.

To increase the objective function, we need to increase the right hand side of constraint c1. It is useless to increase the right hand side of c2.

In [ ]:
(m.c1.uslack(), m.c2.uslack())

Check the objective increases when we add the shadow price to the right hand constraint of c1

In [ ]:
increase_in_c1 = 2
m = model()
m.del_component(m.c1)
m.c1 = pyo.Constraint(rule=m.x1 + 2 * m.x2 <= 10 + increase_in_c1)
solver.solve(m)
assert pyo.value(m.cost) == objective + increase_in_c1 * shadow_price_c1

The reduce cost of a decision variable (i.e. value -13 for variable x2)
is equal to the shadow prive of the non-negativity constraint of the variable
(m.x2 >= 0)

We can check that the objective decrease by the reduce cost of x2.

In [ ]:
m = model()
increase_in_x2 = 1
m.within_x2 = pyo.Constraint(rule=m.x2 >= increase_in_x2)
solver.solve(m)
assert pyo.value(m.cost) < objective
assert pyo.value(m.cost) == objective + increase_in_x2 * reduce_cost_x2

The original cost of x2 in the objective function is too low compared to the cost x1
to make x2 interesting.

The cost of x2 can be increased by -reduce_cost. This is the _opportunity cost_.

It is the amount to add to the coefficient in the objective function for a variable
to become non-zero

In [ ]:
m = model()
m.del_component(m.cost)
epsilon = 1e-6
m.cost = pyo.Objective(expr=10*m.x1 + (7 - reduce_cost_x2 + epsilon)*m.x2, sense = pyo.maximize)
solver.solve(m)
assert pyo.value(m.x2) != 0

We can check that the objective of the dual is equal to the objective of the primal.

Also, after solving the dual:
 - decision variables of the dual are equal to the shadow prices of the primal
 - reduce costs of the dual are equal to slack variables of the primal

# Dual problem

In [ ]:
m_dual = pyo.ConcreteModel()
m_dual.π1 = pyo.Var(within=pyo.NonNegativeReals)
m_dual.π2 = pyo.Var(within=pyo.NonNegativeReals)
m_dual.cost = pyo.Objective(expr=10*m_dual.π1 + 40*m_dual.π2, sense = pyo.minimize)
m_dual.c1 = pyo.Constraint(expr=m_dual.π1 + 3 * m_dual.π2 >= 10)
m_dual.c2 = pyo.Constraint(expr=2 * m_dual.π1 + 4*m_dual.π2 >= 7)
m_dual.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
m_dual.rc = pyo.Suffix(direction=pyo.Suffix.IMPORT)
solver.solve(m_dual)

assert pyo.value(m_dual.cost) == objective
assert pyo.value(m_dual.π1) == shadow_price_c1

The change in the second member or the coefficients of the objective function are only valid within a range.

Original PL in standard form:

$$
\max z = 10 x_1 + 7 x_2 \\
s.t. \\
x_1 + 2 x_2 + u_1 = 10 \\
3 x_1 + 4 x_2 + u_2 = 40 \\
$$

At the end of the resolution, non-basis variables are $u_1$ and $x_2$.


$$
\max z = 100 - 13 x_2 - 10 u_1 \\
s.t. \\
x_1 = 10 - 2 x_2 - u_1 \\
u_2 = 10 + 2 x_2 + 3 u_1 \\
$$

It also gives us the optimal dual basis:
$$
\min w = 100 + 10 x_1 + 10 u_2 \\
   x_2 =  10 -  3 x_1 -    u_2 \\
   u_1 = -13 +  2 x_1 +  2 u_2 
$$

We want to know how much we can change some of the parameters of the PL and the impact of the objective function with changing the variables in basis.

## Sensitivity Analysis for Coefficients of the Objective Function

Let's improve the coefficient of the basis variable $x_1$. We replace its coefficient by $\alpha$: $\max z = \alpha x_1 + 7 x_2 $

We want to keep the same set of basis and non basis variables. Changing the coefficient will not change the equation. We have:
$$
x_1 = 10 - 2 x_2 - u_1 \\
u_2 = 10 + 2 x_2 + 3 u_1 \\
$$

We remplace x_1 by its value in the objective function: 

$$
\max z = \alpha (10 - 2 x_2 - u_1) + 7 x_2\\
\max z = 10 \alpha + (7 - 2 \alpha) x_2 - \alpha u_1\\
$$

The solution doesn't change as long as the reduced costs remains negative.

$$
(7 - 2 * alpha) \le 0 \rightarrow 7 / 2 \le \alpha\\
- alpha \le 0 \rightarrow 0 \le \alpha\\
$$

So, when $\alpha$ is within [7 / 2, $\infty$], objective will increase by $10 * \alpha$

## Sensitivity Analysis for Second Member

Let's improve the second member of the first constraint by $\lambda$: $x_1 + 2 x_2 \le 10 + \lambda$. To the standard form, we have: $x_1 + 2 x_2 + u_1 = 10 + \lambda$

We want to see how much we can increase/decrease 10 by $\lambda$. 
$x_1 + 2 x_2 + (u_1  - \lambda) = 10$. If we define $U_1 = u_1 - \lambda$, the PL will give us the same solution:

$$
\max z = 100 - 13 x_2 - 10 (u_1 - \lambda) = 100 + 10 \lambda - 13 x_2 - 10 u_1  \\
s.t. \\
x_1 = 10 - 2 x_2 - (u_1 - \lambda) \ge 0\\
u_2 = 10 + 2 x_2 + 3 (u_1 - \lambda) \ge 0\\
$$

10 is the shadow price associated to the first constraint.

$u_1$ and $x_2$ must remain non-basis variables, so $u_1 = 0 $ and $x_2 = 0$. So: 

$$
10 + \lambda \ge 0 \rightarrow \lambda \ge -10 \\
10 - 3 \lambda \ge 0 \rightarrow \lambda \le 10 / 3
$$

So, when $\lambda$ is within [-10, 10 / 3], objective will increase by $10 * \lambda$


## Complementary Slackness

Propositions are equivalent:

 * $x_0$, the optimal solution of the primal, and $\pi_0$, the dual one
 * $c^T x_0 = \pi_0^T b$
 * $(\pi_0)_i > 0$ \rightarrow $A_i x_0 = b$
 * $(\pi_0)_i (b - A_i x_0) = 0$ and $(x_0)_j (A^T_i \pi_0 - c_j) = 0$

> It says that if a dual variable is greater than zero (slack) then the corresponding primal constraint must be an equality (tight.) It also says that if the primal constraint is slack then the corresponding dual variable is tight (or zero.)
[CS 435 : Linear Optimization](https://www.cse.iitb.ac.in/~sundar/cs435/lecture15.pdf)